# Giriş
Merhaba arkadaşlar Tensoflow öğrenme serüvenimizde ikinci çalışmadan merhaba, bu çalışmada Quickstart for experts kısmı üzerinde çalışacağız. Aslına bakarsanız ilk çalışmanın benzer adımlarına sahip olacak ama farklı veri seti hazırlamna aşamalarına ve model hazırlama aşamalarını bu bölümde göreceğiz.

In [ ]:
import tensorflow as tf
print("Tensorflow version: ", tf.__version__)
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model


Tensorflow version:  2.15.0


In [ ]:
# Veri seti yükleme
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0 # Normalizasyon kısmı

x_train = x_train[..., tf.newaxis].astype("float32")
x_test = x_test[..., tf.newaxis].astype("float32")

11490434/11490434 [==============================] - 0s 0us/step


**Normalizasyon kısımı** :  Bu kısımda x_train ve x_test içerisinde olan veriler 255.0 değerine gölünerek 0 ile 1 arasına indirgeniyor. Bu durum modelin daha hızlı ve kararlı öğrenmesini sağlıyor.

**x_train = x_train[..., tf.newaxis].astype("float32")** Bu kısımda 28x28 lik resim boyutunu tensor boyutuna çeviriyoruz 28x28x1. Bunu yapma nedenimiz CNN modellerinin tensor veri tipini kabult etmesinden dolayıdır.

In [ ]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(60000, 28, 28, 1)
(10000, 28, 28, 1)
(60000,)
(10000,)


In [ ]:
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32)

test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

**from_tensor_slices** : x_train ve y_train tensorlerinden bir dataset oluşturur.<br>
**shuffle** : Verileri rastgele karıştırır.<br>
**batch** : Verileri 32 öğeden oluşan mini-gruplar haline getirir.<br>

In [ ]:
# Modelin oluşturulması
class MyModel(Model):
  def __init__(self):
    super().__init__()
    self.conv1 = Conv2D(32, 3, activation = "relu") # Katman 1
    self.flatten = Flatten() # Katman 2
    self.d1 = Dense(128, activation = "relu") # katman 3
    self.d2 = Dense(10) # katman 4

  def call(self, x):
    x = self.conv1(x)
    x = self.flatten(x)
    x = self.d1(x)
    return self.d2(x)

model = MyModel()

Model oluşturma kısmı bir kısma bir önce çalışmalar ile aynı ama bazı yeniliklerde mevcur.<br>
**Katman 1** : CNN model yapısından bir katmna 32 filtreye (feature maps) sahiptir, her biri 3x3 boyutunda ve ReLU aktivasyon fonksiyonunu kullanır.<br>

Diğer katmanları ayır ayrı yazmayacağım bir önce çalışmamda katmanlardan bahsetmiştim.
[Bir önceki çalışmam](Gelecek!)

In [ ]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True) # Kayıp(loss) değerlerinin hesaplanma işlemini gerçekleştirir.

optimizer = tf.keras.optimizers.Adam() # Adam optimizer fonksiyonu modeli eğitim adımından sonra güncelemek için kullanılır.

In [ ]:
train_loss = tf.keras.metrics.Mean(name = "train_loss") # Bu eğitim esnasında "loss" eğitim kaybını hesaplayacak metriği oluşturur. train_loss olarak ifade edilecek.
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name = "train_accuracy") # Accuracy değerinin hesaplanacağı kısım. train_accuracy olarak ifade edilecek

test_loss = tf.keras.metrics.Mean(name = "test_loss")
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name = "test_accuracy")

In [ ]:
@tf.function
def train_step(images, labels): # Fonksiyon
  with tf.GradientTape() as tape: # Bu gradient hesaplama bağlamını başlatır. Bu bağlam içindeki tüm operasyonlar, gradientlerin hesaplanması kaydeder.
    predictions = model(images, training = True) # modeli çağırarak görüntüler üzerinde tahminler yapar. training = True modelin eğitim modunda olduğunu belirtir.
    loss = loss_object(labels, predictions) # tahminer ve gerçek etiket arasındaki kaybı hesaplar
  gradients = tape.gradient(loss, model.trainable_variables) # Loss ve model.trainable_variable arasındaki gradientleri hesaplar
  optimizer.apply_gradients(zip(gradients, model.trainable_variables)) # Gradientler ve model değişkenlerini optimize ediciye ileterek modelin parametrelerini günceller.

  train_loss(loss) # eğitim kaybını günceler
  train_accuracy(labels, predictions) # accuracy değerini günceller.

In [ ]:
@tf.function
def test_step(images, labels):
  predictions = model(images, training = False)
  t_loss = loss_object(labels, predictions)
  test_loss(t_loss)
  test_accuracy(labels, predictions)

In [ ]:
epochs = 5

for epoch in range(epochs):
  train_loss.reset_state() # Her adım başında eğitim için metrikler sıfırlanır. Epochlar arası değerlerin ayrılmasını sağlar!
  train_accuracy.reset_state()

  test_loss.reset_state()
  test_accuracy.reset_state()

  for images, labels in train_ds:
    train_step(images, labels) # Eğitim aşaması

  for test_images, test_labels in test_ds:
    test_step(test_images, test_labels) # Test aşması

  print(
      f"Epochs : {epoch + 1}, "
      f"Loss : {train_loss.result():0.2f}, "
      f"Accuracy : {train_accuracy.result() * 100:0.2f}, "
      f"Test Loss: {test_loss.result():0.2f}, "
      f"Test Accuracy: {test_accuracy.result() * 100:0.2f}"
  )

Epochs : 1Loss : 0.13Accuracy : 96.07Test Loss: 0.05Test Accuracy: 98.23
Epochs : 2Loss : 0.04Accuracy : 98.69Test Loss: 0.05Test Accuracy: 98.21
Epochs : 3Loss : 0.02Accuracy : 99.30Test Loss: 0.05Test Accuracy: 98.34
Epochs : 4Loss : 0.01Accuracy : 99.61Test Loss: 0.05Test Accuracy: 98.45
Epochs : 5Loss : 0.01Accuracy : 99.69Test Loss: 0.05Test Accuracy: 98.60


# Sonuç
Quickstart for experts kısmınında sonuna geldik. Bu aşamada beginner seviyesine göre bazı adımları ayrı ayrı kodladık. Fikir vermek için erken diye düşünüyorum beginner seviyesindeki kod ile eğitim yapmak mı? yoksa experts bölümünkdeki kodlar ile eğitim yapma mı? İsteyen istediğini kullansın şimdilik. Tensorflow kütüphanesi tutorial kısmı üzerinde çalışmamızın başındayız ileride bakalım bizi ne gibi süprizler bekliyor hep birlikte göreceğiz!

Aşağıdaki Linklerden beni takip edebilir ve yapacağım çalışmalardan haberdar olabilirsiniz!<br>
[Linkedin](https://www.linkedin.com/in/ihsancenkiz/)<br>
[Github](https://github.com/ihsncnkz)<br>
[Kaggle](https://www.kaggle.com/ihsncnkz)